In [2]:
class_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'y']

In [4]:
print(class_names)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'y']


In [8]:
with open('water_threatment_8x5000.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    print(readCSV)

In [22]:
# Для загрузки данных из csv.
import csv
# Библиотеки машинного обучения.
import tensorflow as tf
from tensorflow import keras
# Библиотека для работы с массивами.
import numpy as np
# Библиотека для построения графиков.
import matplotlib.pyplot as plt


# Зерно для воспроизводимости.
random_state = 42
np.random.seed(random_state)
# TODO: Почему не работает? Исправить!
tf.set_random_seed(random_state)
tf.random.set_random_seed(random_state)


# Функция создания обучающей и тестовой выборок.
def create_feature_sets_and_labels(file, test_size = 0.2):
    # Открытие csv-файла.
    with open(file) as csvfile:
        readCSV = np.genfromtxt(file, delimiter=',')
        readCSV = np.delete(readCSV, 0, 0)
        
        # Строка для отладки.
        # print(readCSV)
        
        np.random.shuffle(readCSV)
        
        # Строка для отладки.
        # print(readCSV)
        
        # Строки для отладки.
        '''
        print(len(readCSV))
        print(readCSV[0:2,0:2])
        for row in readCSV:
            print(row)
        '''
        
        # Деление выборки
        testing_size = int(test_size*len(readCSV))
        train_x = readCSV[:-testing_size,:-1]
        train_y = readCSV[:-testing_size,-1:]
        test_x = readCSV[-testing_size:,:-1]
        test_y = readCSV[-testing_size:,-1:]
    return train_x, train_y, test_x, test_y


# Создание обучающей и тестовой выборок из csv-файла с исходной выборкой.
train_x, train_y, test_x, test_y = create_feature_sets_and_labels('water_threatment_8x5000.csv')

# Строки для отладки.
'''
print()
print(len(train_x))
print(train_x)
print()
print(len(train_y))
print(train_y)
print()
print(len(test_x))
print(test_x)
print()
print(len(test_y))
print(test_y)
'''

# Другой способ задания количества нейронов в слое.
'''
n_neurons = 13
n_nodes_hl1 = n_neurons
n_nodes_hl2 = n_neurons
n_nodes_hl3 = n_neurons
'''

# Количество нейронов в слоях нейронной сети.
n_nodes_hl1 = 11
n_nodes_hl2 = 8
n_nodes_hl3 = 5

# Количество классов - два.
n_classes = 2

# При обучении нейронная сеть будет получать наблюдения пачками.
# Это размер пачки.
batch_size = 500

# Переменные для обучающей и тестовой выборок,
# которые будут инициализированы при обучении нейронной сети.
x = tf.placeholder('float',[None, 8])
y = tf.placeholder('float')


# Задание структуры нейронной сети.
# TODO: Выяснить почему не работает задание зерна для функций tensorflow.
def neural_network_model(data):
    # Задание весов нейронов для 3 скрытых слоев и выходного слоя.
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([8, n_nodes_hl1])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                   'biases':tf.Variable(tf.random_normal([n_classes]))}
    # Описание процесса обучения сети с использованием функции активации RELU.
    # TODO: Попробовать другие функции обучения после того, как работа нейронной сети
    # станет стабильной.
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    # Выдача функцией выходных значений.
    return output


# Функция обучения нейронной сети.
def train_neural_network(x):
    # Получение прогноза нейронной сетью.
    prediction = neural_network_model(x)
    # Вычисление функции потерь.
    # TODO: 1. Подобрать функцию с перекрестной энтропией, более подходящую для
    # бинарной классификации, если такая найдется.
    # TODO: 2. Переделать под совместимость со следующим обновлением библиотеки.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
    # Минимизация функции потерь.
    # TODO: Посмотреть какие еще есть функции минимизации.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    # Количество эпох обучения.
    # TODO: Вынести из функции обучения сети.
    hm_epochs = 100
    # Процесс обучения нейронной сети - работает только тогда,
    # когда вызывается функция обучения.
    with tf.Session() as sess:
        # Инициализация переменных.
        sess.run(tf.initialize_all_variables())
        # Процесс обучения производится заданное количество эпох.
        for epoch in range(hm_epochs):
            # Переменная потерь за эпоху.
            epoch_loss = 0
            # Цикл обучения по пачкам, на случай очень большого количества данных.
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                # Пачки наблюдений.
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                # Расчет и оптимизация функции потерь.
                _, c = sess.run([optimizer, loss], feed_dict={x: batch_x,y: batch_y})
                # Подсчет потерь.
                epoch_loss += c
                # Переход к следующей пачке наблюдений.
                i += batch_size
            
            # Вариант подачи выборки на обучение нейронной сети.
                '''
                for _ in range(int(water_threatment/batch_size)):
                    x, y = 
                '''
            
            # Количество правильных прогнозов.
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            # Доля правильных прогнозов.
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            # Информация об обучении за эпоху.
            print('Epoch {:5.0f}'.format(epoch + 1),
                  'completed out of {:6.0f}.'.format(hm_epochs),
                  'Loss: {:8.3f}.'.format(epoch_loss),
                  'Accuracy: {:1.3f}.'.format(accuracy.eval({x:test_x, y:test_y})))


# Обучение нейронной сети.
train_neural_network(x)

Epoch     1 completed out of    100. Loss:  308.874. Accuracy: 1.000.
Epoch     2 completed out of    100. Loss:  246.267. Accuracy: 1.000.
Epoch     3 completed out of    100. Loss:  186.485. Accuracy: 1.000.
Epoch     4 completed out of    100. Loss:  129.683. Accuracy: 1.000.
Epoch     5 completed out of    100. Loss:   75.926. Accuracy: 0.999.
Epoch     6 completed out of    100. Loss:   27.456. Accuracy: 0.555.
Epoch     7 completed out of    100. Loss:   12.834. Accuracy: 0.635.
Epoch     8 completed out of    100. Loss:   16.590. Accuracy: 0.927.
Epoch     9 completed out of    100. Loss:   17.948. Accuracy: 0.814.
Epoch    10 completed out of    100. Loss:   14.216. Accuracy: 0.723.
Epoch    11 completed out of    100. Loss:   14.713. Accuracy: 0.823.
Epoch    12 completed out of    100. Loss:   15.654. Accuracy: 0.804.
Epoch    13 completed out of    100. Loss:   14.850. Accuracy: 0.781.
Epoch    14 completed out of    100. Loss:   14.939. Accuracy: 0.804.
Epoch    15 complete